In [1]:
#importing necessary libraries
from transformers import AutoProcessor, FlavaModel, FlavaImageModel, AutoImageProcessor, FlavaFeatureExtractor
import numpy as np
from torchmultimodal.models.flava.model import flava_model_for_classification
from transformers import FlavaProcessor, FlavaForPreTraining
import torchvision
import torch
import torch.nn as nn




/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Checking for GPU and settingt he device as GPU if avaialble
if torch.cuda.is_available():
    print("CUDA is available on this system.")
    device = torch.device("cuda")
else:
    print("CUDA is not available on this system.")
    device = torch.device("cpu")




CUDA is available on this system.


In [3]:
print(torch.cuda.memory_summary(device=device, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [4]:
from tqdm import tqdm
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.datasets import MNIST, CUB200

from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
import torchvision.transforms as transforms
from torch.utils.data import random_split


resize_transform = transforms.Resize((224, 224))
grayscale_transform = transforms.Grayscale(num_output_channels=3)
train_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()
    
])

test_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()

])


mnist_train = MNIST(
    './data/mnist', train=True, download=True, transform=train_transform
)

mnist_test = MNIST(
    './data/mnist', train=False, download=True, transform=test_transform
)

# cub_train = CUB200(
#     './data/cub200', train=True, download=True, transform=train_transform
# )

# cub_test = CUB200(
#     './data/cub200', train=False, download=True,  transform=test_transform
# )


In [5]:
#Checking for image shape and size
# Get the first image from the dataset
image, label = mnist_train[0]

# Check the size and shape of the image
size = image.size
shape = image.shape

print(f"Image Size: {size}")
print(f"Image Shape: {shape}")



Image Size: <built-in method size of Tensor object at 0x7fe912826680>
Image Shape: torch.Size([3, 224, 224])


In [6]:
#Classsic class incremental creation from MNIST dataset
# scenario_CIL = nc_benchmark(
#     mnist_train, mnist_test, n_experiences=5, shuffle=True, seed=1234,
#     task_labels=False
# )


#Classsic Domain incremental creation from MNIST dataset
scenario_DIL = ni_benchmark(
    mnist_train, mnist_test, n_experiences=5, shuffle=False, seed=42,
    balance_experiences=True 
)


In [7]:
#creating a FLAVA custom class which calls the pretrained image model (IM encoder only) and returns the features/embeddings instead of a flava basepooling type

class flava_custom(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").to("cuda")

        #self.image_processor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full") 
        #self.model_custom = FlavaModel.from_pretrained("facebook/flava-full").eval().cuda()

        #self.image_processor = FlavaProcessor.from_pretrained("facebook/flava-full", return_codebook_pixels=True)
        #self.model_custom = FlavaForPreTraining.from_pretrained("facebook/flava-full").to("cuda")
        
        
        #self.model_custom = FlavaModel.from_pretrained("facebook/flava-full").eval().to("cuda")
        #self.image_processor = AutoProcessor.from_pretrained("facebook/flava-full")
        self.num_classes = num_classes
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        
        inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
        #with torch.no_grad():
        outputs = self.model_custom(**inputs) 
        
        return outputs.last_hidden_state[:, 0, :]
       



        # score = outputs.last_hidden_state[:, 0, :] 

        # # Passing through a fully connected layer and the number of output classes
        # fc_layer = nn.Linear(in_features=score.size(1), out_features=self.num_classes).to(self.device)

        # # # Get the logits by passing image_features through the fc_layer
        # logits = fc_layer(score).to(self.device)
        
        
        # return logits  #last_hidden_state[:, 0, :]


In [8]:
import torch
image_tensor_with_batch = torch.unsqueeze(mnist_train[0][0], 0)
custom_DIL = flava_custom(num_classes=scenario_DIL.n_classes)
f = custom_DIL.forward(x=image_tensor_with_batch)
print(f.size)
input_ids = torch.argmax(f, dim=1).flatten(0) #important line to convert labels to match dimensions
loss = torch.nn.functional.cross_entropy(f, input_ids)
loss


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


<built-in method size of Tensor object at 0x7fe8a078ce50>


tensor(6.3363, device='cuda:0', grad_fn=<NllLossBackward0>)

Class incremental setting on SplitMNIST using EWC

In [9]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, EWCPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.templates import SupervisedTemplate


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.


custom_DIL = flava_custom(num_classes=10)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    custom_DIL, Adam(custom_DIL.parameters(), lr=0.01, momentum=0.9),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device) 

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

print('Computing accuracy on the whole test set')
# test also returns a dictionary which contains all the metric values
results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 121/121 [04:27<00:00,  2.21s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447197.8037
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447197.8037
	Loss_Epoch/train_phase/train_stream/Task000 = 3.0136
	Loss_MB/train_phase/train_stream/Task000 = 2.3218
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0171
	Time_Epoch/train_phase/train_stream/Task000 = 267.1430
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0973
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2000
100%|██████████| 121/121 [04:25<00:00,  2.19s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447197.8037
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447197.8037
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3808
	Loss_MB/train_phase/train_stream/Task000 = 2.3523
	RunningTime_Epoch/train_phase/train_stream/

EWC on ViT model

In [7]:
import timm
model_vit = timm.create_model('vit_base_patch16_224_in21k', num_classes=10, pretrained=True)

/home/IAIS/jraghu/.local/lib/python3.9/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


In [8]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.



# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    model_vit, SGD(model_vit.parameters(), lr=0.01, momentum=0.9),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

print('Computing accuracy on the whole test set')
# test also returns a dictionary which contains all the metric values
results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 121/121 [02:49<00:00,  1.40s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447257.3984
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447257.3984
	Loss_Epoch/train_phase/train_stream/Task000 = 9.3004
	Loss_MB/train_phase/train_stream/Task000 = 2.2735
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0187
	Time_Epoch/train_phase/train_stream/Task000 = 169.4870
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1025
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2000
100%|██████████| 121/121 [03:03<00:00,  1.52s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447257.3984
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447257.3984
	Loss_Epoch/train_phase/train_stream/Task000 = 2.4527
	Loss_MB/train_phase/train_stream/Task000 = 2.4441
	RunningTime_Epoch/train_phase/train_stream/